##Import

In [ ]:
import numpy as np
import pandas as pd
from google.colab import files
import io
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from google.colab import drive
#https://drive.google.com/file/d/1-S8TXAf33CX7zhyHP_rNNCtX_VueH3-2/view?usp=sharing

##Import preprocessed file from dropbox

In [ ]:
#pd.read_csv('https://drive.google.com/file/d/1-S8TXAf33CX7zhyHP_rNNCtX_VueH3-2/view?usp=sharing')
bkk = 'Bangkok'
ctb = 'Chanthaburi'
cm = 'Chiang Mai'
fire_hotspot = 'Fire hotspot/'
kbr = 'Kanchanaburi'
kk = 'Khon Kaen'
sk = 'Songkhla'
#Chanthaburi_process_data_2
province = [ctb, cm, kbr, bkk, kk, sk]
#Bangkok_process_data_train_new
pm_data_path = [x+'_process_data_train_new_6.csv' for x in province]
path_list = [
             'https://www.dropbox.com/s/70r2zg5rwdpf3nc/Chanthaburi_process_data_train_new_6.csv?dl=1',
             'https://www.dropbox.com/s/km0e3fk7mkokzcl/Chiang%20Mai_process_data_train_new_6.csv?dl=1',
             'https://www.dropbox.com/s/q7j2xum15fr3vz6/Kanchanaburi_process_data_train_new_6.csv?dl=1',
             'https://www.dropbox.com/s/c3ni6n4dszczr3p/Bangkok_process_data_train_new_6.csv?dl=1',
             'https://www.dropbox.com/s/bqh4sjsefgczwms/Khon%20Kaen_process_data_train_new_6.csv?dl=1',
             'https://www.dropbox.com/s/i0etbo2bood5oqx/Songkhla_process_data_train_new_6.csv?dl=1']

In [ ]:
pm_data_train = []
import datetime, pytz
for tmp_path in path_list:
  df = pd.read_csv(tmp_path)
  df['predict_time'] = df['predict_time'].astype('datetime64[ns]').dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok')
  pm_data_train.append(df)

##Data Explaining
_x means past 0 hour, 
_y means past 6 hours,
_x.1 means past 12 hours,
_y.1 means past 18 hours,
_x.2 means past 24 hours,
_y.2 means past 30 hours

For example, if the current time is 12.00 of 10 May 2021, we want to predict from 13.00 of 10 May to 12.00 of 13 May. The data that we use are 6 data points from past 0, 6, 12, 18, 24, 30 hours. The following past hours are 12.00 of 10 May, 06.00 of 10 May, 00.00 of 10 May, and so on.

In [ ]:
pm_data_train[0]

,Unnamed: 0,index,predict_time,target,PM2.5_x,Wind Speed(km/h)_x,lat_x,long_x,Temp(C)_x,hot_spot_score_1_x,hot_spot_score_2_x,hot_spot_score_3_x,datetime_x,PM2.5_y,Wind Speed(km/h)_y,lat_y,long_y,Temp(C)_y,hot_spot_score_1_y,hot_spot_score_2_y,hot_spot_score_3_y,datetime_y,PM2.5_x.1,Wind Speed(km/h),lat,long,Temp(C),hot_spot_score_1,hot_spot_score_2,hot_spot_score_3,datetime_x.1,PM2.5_y.1,datetime_y.1,PM2.5_x.2,datetime_x.2,PM2.5_y.2,datetime_y.2
0,0,0,2016-03-05 13:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
1,1,1,2016-03-05 14:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
2,2,2,2016-03-05 15:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
3,3,3,2016-03-05 16:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
4,4,4,2016-03-05 17:00:00+07:00,34.0,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317803,317803,67,2019-03-07 20:00:00+07:00,15.6,18.7,17.0,12.6096,102.1045,25.2,1112.999598,65.470565,51.445001,2019-03-05 00:00:00+07:00,21.9,21.0,12.6096,102.1045,26.9,1040.281805,49.537229,40.126358,2019-03-04 18:00:00+07:00,16.9,18.0,12.6096,102.1045,30.4,1036.513974,57.584110,43.838731,2019-03-04 12:00:00+07:00,17.8,2019-03-04 06:00:00+07:00,15.2,2019-03-04 00:00:00+07:00,21.0,2019-03-03 18:00:00+07:00
317804,317804,68,2019-03-07 21:00:00+07:00,15.6,18.7,17.0,12.6096,102.1045,25.2,1112.999598,65.470565,51.445001,2019-03-05 00:00:00+07:00,21.9,21.0,12.6096,102.1045,26.9,1040.281805,49.537229,40.126358,2019-03-04 18:00:00+07:00,16.9,18.0,12.6096,102.1045,30.4,1036.513974,57.584110,43.838731,2019-03-04 12:00:00+07:00,17.8,2019-03-04 06:00:00+07:00,15.2,2019-03-04 00:00:00+07:00,21.0,2019-03-03 18:00:00+07:00
317805,317805,69,2019-03-07 22:00:00+07:00,15.6,18.7,17.0,12.6096,102.1045,25.2,1112.999598,65.470565,51.445001,2019-03-05 00:00:00+07:00,21.9,21.0,12.6096,102.1045,26.9,1040.281805,49.537229,40.126358,2019-03-04 18:00:00+07:00,16.9,18.0,12.6096,102.1045,30.4,1036.513974,57.584110,43.838731,2019-03-04 12:00:00+07:00,17.8,2019-03-04 06:00:00+07:00,15.2,2019-03-04 00:00:00+07:00,21.0,2019-03-03 18:00:00+07:00
317806,317806,70,2019-03-07 23:00:00+07:00,17.5,18.7,17.0,12.6096,102.1045,25.2,1112.999598,65.470565,51.445001,2019-03-05 00:00:00+07:00,21.9,21.0,12.6096,102.1045,26.9,1040

###Cleaning the dataframe

In [ ]:
train = pd.concat(pm_data_train)
train.drop(['Unnamed: 0'], axis=1, inplace=True)
train.rename(columns={'index':'number'}, inplace=True)
train.reset_index(inplace=True)
train.drop(['index'], axis=1, inplace=True)
train

,number,predict_time,target,PM2.5_x,Wind Speed(km/h)_x,lat_x,long_x,Temp(C)_x,hot_spot_score_1_x,hot_spot_score_2_x,hot_spot_score_3_x,datetime_x,PM2.5_y,Wind Speed(km/h)_y,lat_y,long_y,Temp(C)_y,hot_spot_score_1_y,hot_spot_score_2_y,hot_spot_score_3_y,datetime_y,PM2.5_x.1,Wind Speed(km/h),lat,long,Temp(C),hot_spot_score_1,hot_spot_score_2,hot_spot_score_3,datetime_x.1,PM2.5_y.1,datetime_y.1,PM2.5_x.2,datetime_x.2,PM2.5_y.2,datetime_y.2
0,0,2016-03-05 13:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
1,1,2016-03-05 14:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
2,2,2016-03-05 15:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
3,3,2016-03-05 16:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
4,4,2016-03-05 17:00:00+07:00,34.0,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906843,67,2019-03-07 20:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00
1906844,68,2019-03-07 21:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00
1906845,69,2019-03-07 22:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00
1906846,70,2019-03-07 23:00:00+07:00,28.1,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,20

###Add other two features which are hour and month of prediction time.

In [ ]:
train['hour'] = train['predict_time'].dt.hour
train['month'] = train['predict_time'].dt.month
train

,number,predict_time,target,PM2.5_x,Wind Speed(km/h)_x,lat_x,long_x,Temp(C)_x,hot_spot_score_1_x,hot_spot_score_2_x,hot_spot_score_3_x,datetime_x,PM2.5_y,Wind Speed(km/h)_y,lat_y,long_y,Temp(C)_y,hot_spot_score_1_y,hot_spot_score_2_y,hot_spot_score_3_y,datetime_y,PM2.5_x.1,Wind Speed(km/h),lat,long,Temp(C),hot_spot_score_1,hot_spot_score_2,hot_spot_score_3,datetime_x.1,PM2.5_y.1,datetime_y.1,PM2.5_x.2,datetime_x.2,PM2.5_y.2,datetime_y.2,hour,month
0,0,2016-03-05 13:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00,13,3
1,1,2016-03-05 14:00:00+07:00,34.2,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00,14,3
2,2,2016-03-05 15:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00,15,3
3,3,2016-03-05 16:00:00+07:00,34.1,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00,16,3
4,4,2016-03-05 17:00:00+07:00,34.0,34.0,12.0,12.6096,102.1045,31.6,616.644576,51.387048,44.229648,2016-03-05 12:00:00+07:00,34.1,18.0,12.6096,102.1045,26.0,1155.703324,64.205740,55.496995,2016-03-05 06:00:00+07:00,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,2016-03-05 00:00:00+07:00,31.1,2016-03-04 18:00:00+07:00,33.1,2016-03-04 12:00:00+07:00,34.9,2016-03-04 06:00:00+07:00,17,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906843,67,2019-03-07 20:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00,20,3
1906844,68,2019-03-07 21:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00,21,3
1906845,69,2019-03-07 22:00:00+07:00,15.9,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,2019-03-04 12:00:00+07:00,26.9,2019-03-04 06:00:00+07:00,28.3,2019-03-04 00:00:00+07:00,19.6,2019-03-03 18:00:00+07:00,22,3
1906846,70,2019-03-07 23:00:00+07:00,28.1,27.4,19.0,7.1988,100.5951,23.5,8.719716,0.458932,0.376579,2019-03-05 00:00:00+07:00,17.7,16.0,7.1988,100.5951,25.1,7.342919,0.458932,0.376579,2019-03-04 18:00:00+07:00,2

In [ ]:
col = list(train.columns)
use_col = [c for c in col if (c != 'predict_time') and (c[:8]!='datetime') and (c not in ['lat_x', 'lat_y', 'long_x', 'long_y'])]
print(use_col)

['number', 'target', 'PM2.5_x', 'Wind Speed(km/h)_x', 'Temp(C)_x', 'hot_spot_score_1_x', 'hot_spot_score_2_x', 'hot_spot_score_3_x', 'PM2.5_y', 'Wind Speed(km/h)_y', 'Temp(C)_y', 'hot_spot_score_1_y', 'hot_spot_score_2_y', 'hot_spot_score_3_y', 'PM2.5_x.1', 'Wind Speed(km/h)', 'lat', 'long', 'Temp(C)', 'hot_spot_score_1', 'hot_spot_score_2', 'hot_spot_score_3', 'PM2.5_y.1', 'PM2.5_x.2', 'PM2.5_y.2', 'hour', 'month']


In [ ]:
train2 = train[use_col].dropna()
train2

,number,target,PM2.5_x,Wind Speed(km/h)_x,Temp(C)_x,hot_spot_score_1_x,hot_spot_score_2_x,hot_spot_score_3_x,PM2.5_y,Wind Speed(km/h)_y,Temp(C)_y,hot_spot_score_1_y,hot_spot_score_2_y,hot_spot_score_3_y,PM2.5_x.1,Wind Speed(km/h),lat,long,Temp(C),hot_spot_score_1,hot_spot_score_2,hot_spot_score_3,PM2.5_y.1,PM2.5_x.2,PM2.5_y.2,hour,month
0,0,34.2,34.0,12.0,31.6,616.644576,51.387048,44.229648,34.1,18.0,26.0,1155.703324,64.205740,55.496995,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,31.1,33.1,34.9,13,3
1,1,34.2,34.0,12.0,31.6,616.644576,51.387048,44.229648,34.1,18.0,26.0,1155.703324,64.205740,55.496995,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,31.1,33.1,34.9,14,3
2,2,34.1,34.0,12.0,31.6,616.644576,51.387048,44.229648,34.1,18.0,26.0,1155.703324,64.205740,55.496995,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,31.1,33.1,34.9,15,3
3,3,34.1,34.0,12.0,31.6,616.644576,51.387048,44.229648,34.1,18.0,26.0,1155.703324,64.205740,55.496995,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,31.1,33.1,34.9,16,3
4,4,34.0,34.0,12.0,31.6,616.644576,51.387048,44.229648,34.1,18.0,26.0,1155.703324,64.205740,55.496995,27.4,23.0,12.6096,102.1045,25.3,819.715205,35.639792,29.525073,31.1,33.1,34.9,17,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906843,67,15.9,27.4,19.0,23.5,8.719716,0.458932,0.376579,17.7,16.0,25.1,7.342919,0.458932,0.376579,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,26.9,28.3,19.6,20,3
1906844,68,15.9,27.4,19.0,23.5,8.719716,0.458932,0.376579,17.7,16.0,25.1,7.342919,0.458932,0.376579,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,26.9,28.3,19.6,21,3
1906845,69,15.9,27.4,19.0,23.5,8.719716,0.458932,0.376579,17.7,16.0,25.1,7.342919,0.458932,0.376579,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,26.9,28.3,19.6,22,3
1906846,70,28.1,27.4,19.0,23.5,8.719716,0.458932,0.376579,17.7,16.0,25.1,7.342919,0.458932,0.376579,24.6,12.0,7.1988,100.5951,29.3,3.962781,0.330232,0.267873,26.9,28.3,19.6,23,3


In [ ]:
train_df = []
#test_df = []
for i in range(72):
  train_df.append(train2[train2.number == i][use_col])
  #test_df.append(test[test.number == i][use_col])
  train_df[i].drop(['number'], axis=1, inplace=True)
  #test_df[i].drop(['number'], axis=1, inplace=True)

In [ ]:
x = [1]*len(train_df[0].columns[1:])
c = train_df[0].columns[1:]
x

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
print(len(x))

25


##Feature Selection
Only use train data and separate into 70% of training and 30% of validation. You can try running but it may use a lot of memory and GPU.

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
model = XGBRegressor(objective='reg:linear', 
                      random_state=3, 
                      max_depth=3,
                      n_estimators=90)
best_score = 10000
for i in range(len(x)):
  
  x[i] = 0
  print(i, x)
  x_tmp = [c[j] for j in range(len(x)) if x[j] == 1]
  my_rms = 0
  for ii in range(72):
    train_df2 = train_df[ii]
    #X_train, y_train = train_df2[train_df2.columns[1:]], train_df2['target'] 
    X_train, y_train = train_df2[x_tmp], train_df2['target']
    X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train, y_train, test_size=0.30, random_state=0)
    model.fit(X_train1, y_train1)
    y_predict = model.predict(X_test1)
    rms = mean_squared_error(y_test1, y_predict, squared=False)
    my_rms += rms
  my_rms /= 72
  if my_rms < best_score:
    best_score = my_rms
    x[i] = 0
  else:
    x[i] = 1
  print(best_score)

0 [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[06:59:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[06:59:31] WARNING: /workspace/sr

KeyboardInterrupt: ignored

###This is the result of feature selection

In [ ]:
#Best 
x = [1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
print(len(x))

25


##Training the model
We use 72 XGBoost models to predict 72 data points, X_train of 72 models are the same, only y_train are different.

In [ ]:
#from sklearn.metrics import mean_squared_error
import pickle
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:linear', 
                      random_state=3, 
                      max_depth=3,
                      n_estimators=90)
x_tmp = [c[j] for j in range(len(x)) if x[j] == 1]
for i in range(72):
  train_df2 = train_df[i]
  X_train, y_train = train_df2[x_tmp], train_df2['target'] 
  bst = model.fit(X_train, y_train)
  name = 'model3_'+str(i)+'.pkl'
  pickle.dump(bst, open(name, "wb"))

[07:00:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:00:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

##Download the 72 models then manually upload to dropbox.

In [ ]:
from google.colab import files
for i in range(18):
  name = 'model3_'+str(i)+'.pkl'
  files.download(name)

In [ ]:
for i in range(18):
  name = 'model3_'+str(i+18)+'.pkl'
  files.download(name)

In [ ]:
name = 'model3_'+str(28)+'.pkl'
files.download(name)

In [ ]:
for i in range(18):
  name = 'model3_'+str(i+36)+'.pkl'
  files.download(name)

In [ ]:
for i in range(18):
  name = 'model3_'+str(i+18*3)+'.pkl'
  files.download(name)